In [0]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)


sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 50
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)



--2019-05-02 15:17:59--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c14::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2019-05-02 15:18:00 (204 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



W0502 15:18:01.494138 139954846914432 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f4911e547b8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


W0502 15:18:01.498202 139954846914432 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f4911e545c0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_14 (Dense)             (None, 24)                1560      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 25        
Total params: 30,129
Trainable params: 30,129
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 6709 samples


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
20000/20000 [==============================] - 315s 16ms/sample - loss: 0.4634 - acc: 0.7720 - val_loss: 0.3911 - val_acc: 0.8228
Epoch 2/50
20000/20000 [==============================] - 312s 16ms/sample - loss: 0.3624 - acc: 0.8324 - val_loss: 0.3876 - val_acc: 0.8243
Epoch 3/50
20000/20000 [==============================] - 312s 16ms/sample - loss: 0.3430 - acc: 0.8437 - val_loss: 0.3811 - val_acc: 0.8280
Epoch 4/50
20000/20000 [==============================] - 311s 16ms/sample - loss: 0.3250 - acc: 0.8550 - val_loss: 0.3835 - val_acc: 0.8202
Epoch 5/50
20000/20000 [==============================] - 313s 16ms/sample - loss: 0.3108 - acc: 0.8618 - val_loss: 0.3855 - val_acc: 0.8256
Epoch 6/50
20000/20000 [==============================] - 312s 16ms/sample - loss: 0.3016 - acc: 0.8666 - val_loss: 0.3812 - val_acc: 0.8299
Epoch 7/50
20000/20000 [==============================] - 311s 16ms/sample - loss: 0.2919 - acc: 0.8710 - val_loss: 0.3831 - val_acc: 0.8286
Epoch 8/50
20

In [1]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'acc')
plot_graphs(history, 'loss')

NameError: ignored

In [0]:
model.save("test.h5")